## Class Structure

In [1]:
class FakeDense:
    def __init__(self, units):
        self.units = units

    def call(self, x):
        return f"Processing  {x} into {self.units} units"

In [6]:
# creating instance
layer = FakeDense(20)
output = layer.call(120)
print(output)

Processing  120 into 20 units


## Adding weights to the Layer

In [7]:
class FakeDense:
    def __init__(self, units):
        self.units = units
        self.weight = 0.5

    def call(self, x):
        return x * self.weight

    def change_weight(self, new_weight):
        self.weight = new_weight

In [8]:
layer = FakeDense(5)

out = layer.call(10)
print(out)

layer.change_weight(3)
out = layer.call(10)
print(out)

5.0
30


## Override a method inside a parent class

In [16]:
class BaseLayer:
    def info(self):
        return print("I'm inside a base layer")

class MyLayer(BaseLayer):   # MyLayer is inheriting from BaseLayer
    def info(self):
        print("I'm a custom layer")

In [19]:
base_layer = BaseLayer()
custom_layer = MyLayer()

base_layer.info()
custom_layer.info()

I'm inside a base layer
I'm a custom layer


## Architecture building (Like custom models)

In [20]:
class MiniModel:
    def __init__(self):
        self.layer1 = FakeDense(10)
        self.layer2 = FakeDense(5)

    def forward(self, x):
        x = self.layer1.call(x)
        x = self.layer2.call(x)
        return x

In [30]:
model = MiniModel()
model.forward(100)

25.0

In [32]:
# activation function

class ReLU:
    def call(self, x):
        return max(0, x)

## Custom Layer mechanics

## Add parameters that update internally

In [35]:
class TrainableLayer:
    def __init__(self):
        self.weight = 1.0

    def call(self, x):
        return x * self.weight

    def update(self, grad):
        self.weight -= grad

In [36]:
layer = TrainableLayer()

out1 = layer.call(10)
print('Output before update:', out1)

layer.update(0.2)

out2 = layer.call(10)
print('Output after update:', out2)

Output before update: 10.0
Output after update: 8.0


## Composition: Layer inside Layer

In [37]:
class Block:
    def __init__(self):
        self.l1 = TrainableLayer()
        self.l2 = TrainableLayer()

    def call(self, x):
        x = self.l1.call(x)
        return self.l2.call(x)

## Writing a manual training step

In [43]:
# prediction = model.forward(x)
# loss = prediction - target
# grad = loss * 0.1
# model.layer1.update(grad)
# model.layer2.update(grad)

## Add callbacks-like behavior

In [44]:
# class
class Callback:
    def on_epoch_end(self, epoch, logs):
        pass

# subclass
class PrintLoss(Callback):
    def on_epoch_end(self, epoch, logs):
        print(f"Epoch {epoch}: Loss = {logs['loss']}")

In [45]:
callback = PrintLoss()

for epoch in range(1,4):
    logs = {"loss": 0.5*epoch}
    callback.on_epoch_end(epoch, logs)

Epoch 1: Loss = 0.5
Epoch 2: Loss = 1.0
Epoch 3: Loss = 1.5


## Mini Deep Learning Framework

### MiniDense Layer

In [47]:
class MiniDense:
    def __init__(self, input_dim, output_dim):
        self.W = [[0] * output_dim for _ in range(input_dim)]
        self.b = [0] * output_dim

    def call(self, x):
        return f"Dense({len(self.W)}->{len(self.b)}) processed {x}"

### ReLU Activation

In [48]:
class ReLU:
    def call(self, x):
        return f"ReLU applied on ({x})"

### MiniModel

In [49]:
class MiniModel:
    def __init__(self):
        self.d1 = MiniDense(2,3)
        self.relu = ReLU()
        self.d2 = MiniDense(3,1)

    def call(self, x):
        x = self.d1.call(x)
        x = self.relu.call(x)
        x = self.d2.call(x)
        return x

### Callback System

In [50]:
class Callback:
    def on_train_begin(self):
        pass

    def on_epoch_end(self, epoch, logs):
        pass

    def on_train_end(self):
        pass

In [51]:
class PrintCallBack(Callback):
    def on_epoch_end(self, epoch, logs):
        print(f"Epoch {epoch} - Output: {logs['output']}")

### Training Loop

In [53]:
class Trainer:
    def __init__(self, model, callbacks=None):
        self.model = model
        self.callbacks = callbacks or []

    def train(self, epochs=5):
        for cb in self.callbacks:
            cb.on_train_begin()

        for epoch in range(epochs):
            output = self.model.call('input_data')

            logs = {'output':output}

            for cb in self.callbacks:
                cb.on_epoch_end(epoch, logs)

        for cb in self.callbacks:
            cb.on_train_end()

In [54]:
model = MiniModel()
callback = PrintCallBack()
trainer = Trainer(model, callbacks=[callback])

trainer.train(epochs=3)

Epoch 0 - Output: Dense(3->1) processed ReLU applied on (Dense(2->3) processed input_data)
Epoch 1 - Output: Dense(3->1) processed ReLU applied on (Dense(2->3) processed input_data)
Epoch 2 - Output: Dense(3->1) processed ReLU applied on (Dense(2->3) processed input_data)
